In [79]:
import pandas as pd
import random
from pathlib import Path
import numpy as np
import psycopg2

# Данные

## Подготовка данных для таблицы book

In [4]:
directory = Path("input/book")
book = pd.concat([pd.read_csv(f).drop(
    ['RatingDist1', 'RatingDist4', 'RatingDistTotal', 'RatingDist2', 'RatingDist5', 'RatingDist3', 'Id'], axis=1) for f
    in directory.glob("*.csv")], ignore_index=True)

# book = book.drop(['pagesNumber', 'Count of text reviews'], axis=1)
book.Language = book.Language.fillna('eng')
book.pagesNumber = book.pagesNumber.fillna(random.randint(250, 1500))
book.pagesNumber = book.pagesNumber.astype('int')
book.Publisher = book.Publisher.fillna('Alpina')
book['Genre'] = np.random.choice(
    ['Action and Adventure', 'Classics', 'Comic Book', 'Detective', 'Mystery', 'Fantasy', 'Historical Fiction',
     'Literary Fiction', 'Horror', 'Romance', 'Science Fiction', 'Short Stories', 'Suspense', 'Thrillers',
     'Biographies', 'Autobiographies', 'Cookbooks', 'Essays', 'History', 'Memoir', 'Poetry', 'Self-Help', 'True Crime'],
    len(book))
book.ISBN = book.ISBN.fillna(random.randint(1000, 10000000))

book.loc[book['PublishYear'] > 2023, 'PublishYear'] = random.randint(1900, 2023)
book.loc[book['PublishMonth'] > 12, 'PublishMonth'] = random.randint(1, 12)
book.loc[book['PublishDay'] > 31, 'PublishDay'] = random.randint(1, 31)
book.loc[(book['Language'] == "en-US") | (book['Language'] == "en-GB") | (book['Language'] == "en-CA") | (
        book['Language'] == "--"), 'Language'] = "eng"

book = book[:1000]

In [5]:
# book.isnull().sum()

## Данные для таблиц
- book
- book_genre
- book_lang
- book_publisher
- book_author

In [6]:
Book_genre = pd.DataFrame(data=book.Genre.unique(), columns=['Genre'])
Book_lang = pd.DataFrame(data=book.Language.unique(), columns=['Language'])
Book_publisher = pd.DataFrame(data=book.Publisher.unique(), columns=['Publisher'])
Book_author = pd.DataFrame(data=book.Authors.unique(), columns=['Authors'])

columns = [[Book_genre, "Genre"], [Book_lang, "Language"], [Book_publisher, "Publisher"], [Book_author, "Authors"]]
for col in columns:
    for index, string in enumerate(col[0].iloc[:, 0]):
        book.loc[book[col[1]] == string, col[1]] = index + 1

book.columns = ['book_title', 'book_page_num', 'book_publish_month', 'book_publish_day', 'book_publisher',
                'book_num_reviews', 'book_publish_year', 'book_lang', 'book_author', 'book_rating', 'book_isbn',
                'book_genre']

In [308]:
Book_genre.to_csv("output/book_genre.csv", index=False)
Book_lang.to_csv("output/book_lang.csv", index=False)
Book_publisher.to_csv("output/book_publisher.csv", index=False)
Book_author.to_csv("output/book_author.csv", index=False)

book.to_csv("output/book.csv", index=False)

In [85]:
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="library",
    user='admin',
    password='admin')

cur = conn.cursor()

In [88]:
address = pd.read_csv("input/address/open_pubs.csv").drop(["fas_id", "name", "local_authority", "easting", "northing"], axis=1)

# address = address[:1000]

# address = address.values.tolist()
# address_list = list()
# for elem in address:
#     address_list.append(f"'{elem[0]}', '{elem[1]}', st_makepoint({elem[2]}, {elem[3]})")
#
# cur.execute(f"INSERT INTO address (address_name, address_postcode, address_geo) VALUES ({'), ('.join(address_list)});")

address.to_csv("output/address.csv", index=False)

In [87]:
conn.commit()